In [ ]:
# account to access namespaces and pods list and details
kubectl create serviceaccount cp-api-explorer

# ro permissions - GET/LIST/WATCH namespaces and pods on cluster level
kubectl create clusterrole ns-reader --verb=get,list,watch --resource=namespaces,pods

# bind role(permissions) to account
kubectl create clusterrolebinding ns-reader --clusterrole ns-reader --serviceaccount default:cp-api-explorer


[doc](https://nieldw.medium.com/curling-the-kubernetes-api-server-d7675cfc398c)

In [ ]:
# account has secret(s)
kubectl get serviceaccount cp-api-explorer -o json

In [ ]:
# list secrets
kubectl get secrets

In [ ]:
# we know which from SA, right
$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
gc $caCrtFilename

In [ ]:
# k8s API endpoint (if not in minikube, right?)
$server = kubectl -n default get endpoints kubernetes --no-headers | awk '{ print $2 }'
$apiServerUrl = "https://$($server)"
$apiServerUrl

In [ ]:
kubectl config view -o json | jq '.clusters[] | .name'

# kubectl config view -o json | jq '.clusters[] | select(.name == ""minikube"")'

kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

In [ ]:
# run few servers
1..10 | % { kubectl run "web-$($_)" --image nginx }

In [ ]:
# what does one pod look like?
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[0]'

{
  "metadata": {
    "name": "web-1",
    "namespace": "default",
    "uid": "fc775b3c-5df9-473a-9146-4fce97181fdb",
    "resourceVersion": "1130",
    "creationTimestamp": "2022-03-30T17:08:32Z",
    "labels": {
      "run": "web-1"
    },
    "managedFields": [
      {
        "manager": "kubectl-run",
        "operation": "Update",
        "apiVersion": "v1",
        "time": "2022-03-30T17:08:32Z",
        "fieldsType": "FieldsV1",
        "fieldsV1": {
          "f:metadata": {
            "f:labels": {
              ".": {},
              "f:run": {}
            }
          },
          "f:spec": {
            "f:containers": {
              "k:{\"name\":\"web-1\"}": {
                ".": {},
                "f:image": {},
                "f:imagePullPolicy": {},
                "f:name": {},
                "f:resources": {},
                "f:terminationMessagePath": {},
                "f:terminationMessagePolicy": {}
              }
            },
            "f:dnsPolicy":

In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .metadata.name'

"web-1"
"web-10"
"web-2"
"web-3"
"web-4"
"web-5"
"web-6"
"web-7"
"web-8"
"web-9"
"web01"


In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .status.podIPs[0].ip'

"172.17.0.5"
"172.17.0.13"
"172.17.0.4"
"172.17.0.7"
"172.17.0.6"
"172.17.0.10"
"172.17.0.8"
"172.17.0.9"
"172.17.0.11"
"172.17.0.12"
"172.17.0.3"
